## Imports

In [1]:
import io.hops.util.Hops
import scala.collection.JavaConversions._
import collection.JavaConverters._

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
45,application_1545231463715_0065,spark,idle,Link,Link,✔


SparkSession available as 'spark'.
import io.hops.util.Hops
import scala.collection.JavaConversions._
import collection.JavaConverters._


## Get Project Featurestore

Each project with the featurestore enabled gets its own Hive database for the featurestore, the name of the featurestore database is 'projectname_featurestore' and can be retrieved from the hops-util-py featurestore API

In [ ]:
Hops.getProjectFeaturestore

## Get all Featurestores Accessible in the Current Project

Feature stores can be shared across projects just like other Hopsworks datasets. You can use this API function to list all the featurestores accessible in the project programmatically.

In [ ]:
Hops.getProjectFeaturestores

## Get Individual Feature

When retrieving a single feature from the featurestore, the hops-util-py library will infer which featuregroup the feature belongs to by querying the metastore, but you can also explicitly specify which featuregroup and version to query. If there are multiple features of the same name in the featurestore, it is required to specify enough information to uniquely identify the feature (e.g which featuregroup and which version).  If no featurestore is provided it will default to the project's featurestore.

Without specifying featuregroup:

In [ ]:
Hops.getFeature(spark, "action", Hops.getProjectFeaturestore).show(5)

With specifed featuregroup and version:

In [ ]:
Hops.getFeature(spark, "action", Hops.getProjectFeaturestore, "web_logs_features", 1).show(5)

## Get Featuregroup

You can get an entire featuregroup from the API. If no featurestore is provided the API will default to the project's featurestore, if no version is provided it will default to version 1 of the featuregroup.

In [ ]:
Hops.getFeaturegroup(spark, "trx_summary_features", Hops.getProjectFeaturestore, 1).show(5)

## Get Set of Features

When retrieving a list of features from the featurestore, the hops-util-py library will infer which featuregroup the features belongs to by querying the metastore. If the features reside in different featuregroups, the library will also **try** to infer how to join the features together based on common columns. If the JOIN query cannot be inferred due to existence of multiple features with the same name or non-obvious JOIN query, the user need to supply enough information to the API call to be able to query the featurestore. If the user already knows the JOIN query it can also run `Hops.queryFeaturestore(joinQuery)` directly (an example of using this approach is shown further down in this notebook). If no featurestore is provided it will default to the project's featurestore.

In [ ]:
val features = List("pagerank", "triangle_count", "avg_trx")

In [ ]:
Hops.getFeatures(spark, features, Hops.getProjectFeaturestore).show(5)

Without specifying the join key but specifying featuregroups:

In [ ]:
val featuregroupsMap = Map[String, Integer]("trx_graph_summary_features"->1,"trx_summary_features"->1)
val javaFeaturegroupsMap = new java.util.HashMap[String, Integer](featuregroupsMap)

In [ ]:
Hops.getFeatures(spark, features, Hops.getProjectFeaturestore, javaFeaturegroupsMap).show(5)

Specifying both featuregroups and join key:

In [ ]:
Hops.getFeatures(spark, features, Hops.getProjectFeaturestore, javaFeaturegroupsMap, "cust_id").show(5)

### Advanced examples

Getting 10 features from two different featuregroups without specifying the featuregroups

In [ ]:
val features1 = List("pagerank", "triangle_count", "avg_trx", "count_trx", "max_trx", "min_trx", "balance", "birthdate", "join_date", "number_of_accounts")
Hops.getFeatures(spark, features1, Hops.getProjectFeaturestore).show(5)

If you try to get features that exist in multiple featuregroups, the library will not be able to infer from which featuregroup to get the features, so you must specify the featuregroups explicitly as an argument

In [ ]:
val features2 = List("pagerank", "triangle_count", "avg_trx", "count_trx", "max_trx", "min_trx", "balance", "birthdate", "join_date", "number_of_accounts", "pep")
Hops.getFeatures(spark, features2, Hops.getProjectFeaturestore).show(5)

If we specify the featuregroup to get the feature that exists in multiple featuregroups, the library can infer how to get the features:

In [ ]:
val featuregroupsMap1 = Map[String, Integer](
    "trx_graph_summary_features"->1,
    "trx_summary_features"->1,
    "demographic_features" ->1
)
val javaFeaturegroupsMap1 = new java.util.HashMap[String, Integer](featuregroupsMap1)
Hops.getFeatures(spark, features2, Hops.getProjectFeaturestore, javaFeaturegroupsMap1).show(5)

Example of getting 19 features from 5 different featuregroups:

In [ ]:
val features3 = List("pagerank", "triangle_count", "avg_trx", "count_trx", "max_trx", "min_trx",
    "balance", "birthdate", "join_date", "number_of_accounts", "pep", "customer_type", "gender", "web_id",
    "time_spent_seconds", "address", "action", "report_date", "report_id")
val featuregroupsMap2 = Map[String, Integer](
    "trx_graph_summary_features"->1,
    "trx_summary_features"->1,
    "demographic_features" ->1,
    "web_logs_features" -> 1,
    "police_report_features" -> 1
)
val javaFeaturegroupsMap2 = new java.util.HashMap[String, Integer](featuregroupsMap2)
Hops.getFeatures(spark, features3, Hops.getProjectFeaturestore, javaFeaturegroupsMap2).show(5)

Sometimes you might want to get a feature that exist in multiple featuregroups and you want to include all of these featuregroups in your query, then you can specify from which of the featuregroup to get the feature by prepending the feature-name with the featuregroup name + '_version', e.g: 'demographic_features_1.cust_id'. If you don't specify this the query will fail as the library won't know from which of your specified featuregroups to get the feature:

In [ ]:
val features4 = List("pagerank", "triangle_count", "avg_trx", "count_trx", "max_trx", "min_trx",
    "balance", "birthdate", "join_date", "number_of_accounts", "pep", "customer_type", "gender", "web_id",
    "time_spent_seconds", "address", "action", "report_date", "report_id", "cust_id")
Hops.getFeatures(spark, features4, Hops.getProjectFeaturestore, javaFeaturegroupsMap2).show(5)

If we change 'cust_id' to 'featuregroupname_version.cust_id' the library knows where to get the feature from and the query works:

In [ ]:
val features5 = List("pagerank", "triangle_count", "avg_trx", "count_trx", "max_trx", "min_trx",
    "balance", "birthdate", "join_date", "number_of_accounts", "pep", "customer_type", "gender", "web_id",
    "time_spent_seconds", "address", "action", "report_date", "report_id", "demographic_features_1.cust_id")
Hops.getFeatures(spark, features5, Hops.getProjectFeaturestore, javaFeaturegroupsMap2).show(5)

## Free Text Query from Feature Store

For complex queries that cannot be inferred by the helper functions, enter the sql directly to the method `Hops.queryFeaturestore()` it will default to the project specific feature store but you can also specify it explicitly.

Without specifying the featurestore it will default to the project-specific featurestore:

In [ ]:
Hops.queryFeaturestore(
    spark,
    "SELECT * FROM trx_graph_summary_features_1 WHERE triangle_count > 5",
    null
).show(5)

You can also specify the featurestore to query explicitly:

In [ ]:
Hops.queryFeaturestore(
    spark,
    "SELECT * FROM trx_graph_summary_features_1 WHERE triangle_count > 5",
    Hops.getProjectFeaturestore
).show(5)

## Write to the Feature Store

Lets first get some sample data to insert

In [ ]:
val sampleDataMap = Map("hops_customer_1"-> 3, "hops_customer_2"-> 4)
val sampleDataDf = sampleDataMap.toSeq.toDF("customer_type", "id")

In [ ]:
sampleDataDf.show()

Lets inspect the contents of the featuregroup 'customer_type_lookup' that we are going to insert the sample data into

In [ ]:
val sparkDf = Hops.getFeaturegroup(spark, "customer_type_lookup", Hops.getProjectFeaturestore, 1)

In [ ]:
sparkDf.show()

In [ ]:
sparkDf.count()

Now we can insert the sample data and verify the new contents of the featuregroup. By default the insert mode is "append", the featurestore is the project's featurestore and the version is 1 (the statistics part will be covered later in the notebook)

In [ ]:
val featuregroup = "customer_type_lookup"
val featurestore = Hops.getProjectFeaturestore 
val featuregroupVersion = 1 
val mode = "append"
val descriptiveStats = false
val featureCorr = false
val featureHistograms = false
val clusterAnalysis = false
val statColumns = List[String]().asJava
val numBins = null
val corrMethod = null
val numClusters = null
val description = "trx_summary_features without the column count_trx"

In [ ]:
Hops.insertIntoFeaturegroup(
    sampleDataDf, 
    spark, 
    featuregroup,
    featurestore,
    featuregroupVersion,
    mode,
    descriptiveStats, 
    featureCorr,
    featureHistograms, 
    clusterAnalysis, 
    statColumns, 
    numBins,
    corrMethod, 
    numClusters
)

In [ ]:
Hops.getFeaturegroup(spark, "customer_type_lookup", Hops.getProjectFeaturestore, 1).show()

In [ ]:
Hops.getFeaturegroup(spark, "customer_type_lookup", Hops.getProjectFeaturestore, 1).count

The two supported insert modes are "append" and "overwrite"

In [ ]:
val mode = "overwrite"

In [ ]:
Hops.insertIntoFeaturegroup(
    sampleDataDf, 
    spark, 
    featuregroup,
    featurestore,
    featuregroupVersion,
    mode,
    descriptiveStats, 
    featureCorr,
    featureHistograms, 
    clusterAnalysis, 
    statColumns, 
    numBins,
    corrMethod, 
    numClusters
)

In [ ]:
Hops.getFeaturegroup(spark, "customer_type_lookup", Hops.getProjectFeaturestore, 1).show()

In [ ]:
Hops.getFeaturegroup(spark, "customer_type_lookup", Hops.getProjectFeaturestore, 1).count

## Create a Featuregroup From a Spark Dataframe

In most cases it is recommended that featuregroups are created in the UI on Hopsworks and that care is taken in documenting the featuregroup. However, sometimes it is practical to create a featuregroup directly from a spark dataframe and fill in the metadata about the featuregroup later in the UI. This can be done through the create_featuregroup API function.

Lets create a new featuregroup that contains the same contents as the featuregroup trx_summary except the the column count_trx is dropped

In [ ]:
val trxSummaryDf = Hops.getFeaturegroup(spark, "trx_summary_features", Hops.getProjectFeaturestore, 1)
val trxSummaryDf1 = trxSummaryDf.drop("count_trx")

In [ ]:
trxSummaryDf1.show(5)

When a feature group is created you can specify metadata about the feature group or set it to null and fill it in later in the feature registry UI. (The statistics part will be explained later on in this notebook)

In [ ]:
val jobId = null
val dependencies = List[String]().asJava
val primaryKey = null
val descriptiveStats = false
val featureCorr = false
val featureHistograms = false
val clusterAnalysis = false
val statColumns = List[String]().asJava
val numBins = null
val corrMethod = null
val numClusters = null
val description = "trx_summary_features without the column count_trx"

Lets now create a new featuregroup using the transformed dataframe

In [ ]:
Hops.createFeaturegroup(
    spark, trxSummaryDf1, "trx_summary_features_2", Hops.getProjectFeaturestore,
    1, description, jobId,
    dependencies, primaryKey, descriptiveStats, featureCorr,
      featureHistograms, clusterAnalysis, statColumns, numBins,
      corrMethod, numClusters)

## Compute Featuregroup Statistics

Statistics about a featuregroup can be useful in the stage of feature engineering and when deciding which features to use for training.

To compute statistics about an existing featuregroup (that should not be empty of course), you can use the API call update_featuregroup_stats. By default it will compute all statistics (descriptive, feature correlation, histograms, and cluster analysis), use the project's featurestore, use version 1 of the featuregroup and use all columns for computing statistics:

In [ ]:
val featuregroup = "trx_summary_features"
val featurestore = Hops.getProjectFeaturestore
val featuregroupVersion = 1
val descriptiveStats = true
val featureCorr = true
val featureHistograms = true
val clusterAnalysis = true
val statColumns = null // null means all columns will be used
val numBins = 20
val corrMethod = "pearson"
val numClusters = 5

In [ ]:
Hops.updateFeaturegroupStats(
    spark, featuregroup, Hops.getProjectFeaturestore, featuregroupVersion,
    descriptiveStats, featureCorr, featureHistograms, clusterAnalysis, statColumns,
    numBins, corrMethod, numClusters
)

## Create Managed Training Datasets From Sets of Features

After you have found the features you need in the featurestore you can materialize the features into a training dataset so that you can train a machine learning model using the features. Just as for featuregroups, it is useful to version and document training datasets, for this reason HopsML supports **managed training datasets** which enables you to easily version, document and automate the materialization of training datasets.

Metadata for a training dataset can be created from the Hopsworks UI or directly from the API with the function create_training_dataset. The training datasets in a project are stored in a top-level dataset called Training_Datasets, (i.e `hdfs:///Projects/<ProjectName>/Training_Datasets`.

Once a training dataset have been created you can find it in the featurestore UI in hopsworks under the tab Training datasets, from there you can also edit the metadata if necessary. After a training dataset have been created with the necessary metadata you can save the actual data in the training dataset by using the API function insert_into_training_dataset.

Lets create a dataset called `AML_dataset` by using a set of relevant features from the featurestore.

First we select the features (and/or labels) that we want

In [ ]:
val features = List("pagerank", 
                    "triangle_count", 
                    "avg_trx", 
                    "count_trx", 
                    "max_trx", 
                    "min_trx", 
                    "balance", 
                    "number_of_accounts", 
                    "pep")
val featuregroupsToVersionMap = Map[String, Integer](
    "trx_graph_summary_features"->1,
    "trx_summary_features"->1,
    "demographic_features" ->1
)
val javaFeaturegroupsMap = new java.util.HashMap[String, Integer](featuregroupsToVersionMap)

In [ ]:
val datasetDf = Hops.getFeatures(spark, features, Hops.getProjectFeaturestore, javaFeaturegroupsMap)

In [ ]:
datasetDf.show(5)

When a training dataset is created you can specify metadata about the training dataset or set it to null and fill it in later in the feature registry UI.

In [ ]:
val trainingDatasetName = "AML_dataset"
val jobId = null
val dependencies = List[String]().asJava
val primaryKey = null
val dataFormat = "tfrecords"
val descriptiveStats = false
val featureCorr = false
val featureHistograms = false
val clusterAnalysis = false
val statColumns = List[String]().asJava
val numBins = null
val corrMethod = null
val numClusters = null
val description = "Dataset with features for training an AML model"

In [ ]:
Hops.createTrainingDataset(
    spark, datasetDf, trainingDatasetName, Hops.getProjectFeaturestore,
    1, description, jobId, dataFormat,
    dependencies, descriptiveStats, featureCorr,
      featureHistograms, clusterAnalysis, statColumns, numBins,
      corrMethod, numClusters)

In [ ]:
val trainingDatasetName = "TestDataset"
val jobId = null
val dependencies = List[String]().asJava
val primaryKey = null
val dataFormat = "csv"
val descriptiveStats = false
val featureCorr = false
val featureHistograms = false
val clusterAnalysis = false
val statColumns = List[String]().asJava
val numBins = null
val corrMethod = null
val numClusters = null
val description = "Dataset for Demo purposes"

In [ ]:
Hops.createTrainingDataset(
    spark, datasetDf, trainingDatasetName, Hops.getProjectFeaturestore,
    1, description, jobId, dataFormat,
    dependencies, descriptiveStats, featureCorr,
      featureHistograms, clusterAnalysis, statColumns, numBins,
      corrMethod, numClusters)

## Inserting Into an Existing Training Dataset

Once a dataset have been created, its metadata is browsable in the featurestore registry in the Hopsworks UI. If you don't want to create a new training dataset but just overwrite new data into an existing training dataset (training datasets are immutable and generally stored in binary formats, modifying an existing traning dataset is not supported), you can use the API function `insertIntoTrainingDataset`

In [ ]:
val trainingDataset = "TestDataset"
val featurestore = Hops.getProjectFeaturestore 
val trainingDatasetVersion = 1 
val mode = "append"
val descriptiveStats = false
val featureCorr = false
val featureHistograms = false
val clusterAnalysis = false
val statColumns = List[String]().asJava
val numBins = null
val corrMethod = null
val numClusters = null
val description = "trx_summary_features without the column count_trx"

In [ ]:
Hops.insertIntoTrainingDataset(
    datasetDf, 
    spark,
    trainingDataset,
    featurestore,
    trainingDatasetVersion,
    descriptiveStats, 
    featureCorr,
    featureHistograms, 
    clusterAnalysis, 
    statColumns, 
    numBins,
    corrMethod, 
    numClusters)

## Get Training Dataset Path

After a **managed** dataset have been created, it is easy to share it and re-use it for training various models. For example if the dataset have been materialized in tf-records format you can call the method `getTrainingDatasetPath(training_dataset)` to get the HDFS path and read it directly in your tensorflow/keras/pytorch code. By default the library will look for the training dataset in the project's featurestore and use version 1

In [2]:
Hops.getTrainingDatasetPath("AML_dataset", Hops.getProjectFeaturestore, 1)

res1: String = /Projects/fs_demo/Training_Datasets/AML_dataset_1/AML_dataset


## Get Featurestore Metadata

To explore the contents of the featurestore we recommend using the featurestore page in the Hopsworks UI but you can also get the metadata programmatically from the REST API

### List all Feature Stores Accessible In the Project

In [ ]:
Hops.getProjectFeaturestores()

### List all Feature Groups in a Feature Store

In [ ]:
Hops.getFeaturegroups(Hops.getProjectFeaturestore)

### List all Training Datasets in a Feature Store

In [ ]:
Hops.getTrainingDatasets(Hops.getProjectFeaturestore)

### Get All Metadata (Features, Feature groups, Training Datasets) for a Feature Store

In [ ]:
Hops.getFeaturestoreMetadata(Hops.getProjectFeaturestore)